<a href="https://colab.research.google.com/github/tinamilo6/Textmining/blob/main/A03_TextMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pandas sklearn nltk

# Assignment 3 - Text Mining

Project management and tools for health informatics

## 1. Download and prepare data:

**Do not alter the code in this Section!**

The code in this section downloads the [IMDB IMDB Large Movie Review Dataset]('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz') which is the dataset you will be working on in this assignment.

In [2]:
import os
import tarfile
from urllib.request import urlretrieve

In [3]:
if not os.path.exists('aclImdb'):
    # download data:
    urlretrieve('https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz', 'aclImdb.tar.gz')

    # unzip data:
    with tarfile.open('aclImdb.tar.gz') as file:
        file.extractall('./')

## 2. Some helper Functions:

**Do not alter the code in this Section!**

This section contains the code for some helper functions that will be useful for solving the assignment. Example code on how to use the functions is provided in section 3.

In [4]:
import pandas as pd

from typing import Literal, Tuple, Iterable

Function for loading data into a pandas dataframe:

In [5]:
def load_data(split:Literal['train', 'test'], texts_per_class:int=500) -> pd.DataFrame:
    ''' Loads the data into a pandas dataframe.'''
    paths  = []
    labels = []

    for label in ('pos', 'neg'):
        # get all files in the folder:
        files = os.listdir(os.path.join('aclImdb', split, label))[:texts_per_class]

        # append them to the lists:
        paths.extend([os.path.join('aclImdb', split, label, f) for f in files])
        labels.extend([label] * len(files))

    return pd.DataFrame({'path':paths, 'label':labels})

Function for loading a specific text:

In [6]:
def load_text(path:str) -> str:
    ''' Reads a single text given the path. '''
    # read file from disk:
    with open(path, 'r', encoding='utf8') as file:
        s = file.read()

    return s

Function for iterating through multiple texts:

In [7]:
def iterate_texts(data:pd.DataFrame) -> Iterable[Tuple[str, str]]:
    ''' Iterates through a pandas dataframe. '''

    for path in data['path'].values:
        # read file from disk:
        with open(path, 'r', encoding='utf8') as file:
            text = file.read()

        yield text

## 3. Text Mining Pipeline

This section will cover the text mining steps for this assignment. The following steps will be performed:

1. **Analyze the Data for Difficult Parts**  
   - Reviewing the data to identify challenging aspects such as contractions, informal language, and complex sentence structures.

2. **Replace Contractions and Informal Language**  
   - Expanding common contractions and replacing informal phrases to standardize the text for processing.
   - This part is optinal and it should be later evaluated with normalization and without.

3. **Tokenize the Texts**  
   - Applying a tokenizer to break down the text into individual tokens for analysis.

4. **Training and Predicting with ML**
   - Training different ML models with the tokenized data and predicting the test data to evaluate the accuracy.

### Import needed libraries

In [8]:
# Import needed libaries for the preparation of the texts
import random

### Load the training data
The loaded data frpom the zip file should be saved into a `data_train` and `data_test` DataFrame.
These can be further on be used to access the train and test data.

In [9]:
data_train = load_data('train')
data_test  = load_data('test')
data_train

,path,label
0,aclImdb\train\pos\0_9.txt,pos
1,aclImdb\train\pos\10000_8.txt,pos
2,aclImdb\train\pos\10001_10.txt,pos
3,aclImdb\train\pos\10002_7.txt,pos
4,aclImdb\train\pos\10003_8.txt,pos
...,...,...
995,aclImdb\train\neg\10446_2.txt,neg
996,aclImdb\train\neg\10447_1.txt,neg
997,aclImdb\train\neg\10448_1.txt,neg
998,aclImdb\train\neg\10449_4.txt,neg


### 1. Assess the data / texts for difficult parts
In this part, sample texts are printed and then analyzed for diffult parts, that could affect the text mining process.

In [10]:
# Number of samples to be printed
n_samples = 7

# Randomly sample indices from "data_train"
sample_indices = random.sample(range(len(data_train)), n_samples)

# Load and print each sample using the "load_text" function
for i, id in enumerate(sample_indices, start=1):
    # Load text from the file path specified in 'path' column
    text = load_text(data_train.loc[id, 'path'])
    print(f"Sample {i}:\n")
    print(text)
    print("\n" + "="*80 + "\n")

Sample 1:

The Haunted World of Edward D. Wood, Jr. isn't a particularly good documentary. Aesthetically, it's lackluster and cheap looking, the people in it go off on tangents which make it very unfocused and in-cohesive, but this adds to it's charm. I say this because it's a documentary about an oddball that made oddball pictures and surrounded himself with fellow oddballs and, as such, there's really no other way to document the life and career of the man and his crew of misfits. There are some glimpses of insight into both the genius and the ineptness of Wood, and the portrayal of both qualities is a credit to the genuineness of the documentary. Overall, it's worth a watch for the Wood fan and those of cinema in general, but don't expect brilliance here. Expect a documentary made after Wood's own heart.


Sample 2:

Lorenzo Lamas stars as some type of CIA agent, who captures some exotic beauty named Alexa, kidnaps her daughter and forces her to fight her former employers. O.J Simps

## Limitations/Issues Captured from the Above Text Samples

- **HTML tags and special characters**
- **Punctuation and symbols** (e.g., `&`)
- **Contractions** (e.g., "isn't", "I'll", "I'm")
- **Parentheses and annotations** (e.g., "(Crouching Tiger)")
- **Informal formatting** (e.g., "my rating is ****")
- **Ambiguity and polysemy** (e.g., "dictators", "nuts")
- **Long and complex sentences**
- **Informal language** (e.g., "what can be so bad about that?")
- **Quotation marks** (e.g., "dictators", "sin")

We will elaborate on these issues and add our conclusions in the report.

### 2. Preprocessing: Simplify the Text

This section focuses on preprocessing the texts to make them more suitable for text mining. Identified constraints from the analysis step will be addressed and, as far as possible, eliminated to improve processing accuracy.


In [11]:
import re

def remove_html_tags(text):
    # Regex to match HTML tags
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text



In [12]:
import string

def remove_punctuation_and_symbols(text):
    # Remove punctuation and special symbols using regex
    clean_text = re.sub(r'[^\w\s]', '', text)
    return clean_text



In [13]:
def remove_parentheses(text):
    # Remove text inside parentheses along with parentheses
    clean_text = re.sub(r'\(.*?\)', '', text)
    return clean_text

#### Expand the contractions
There are python libraries that focus on expanding contractions. For easier loading (if these packages are not available) we decided to create our own replacement list.

In [14]:
# Dictionary of common contractions and their expanded forms
contractions_dict = {
    "i'm": "I am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "i've": "I have",
    "you've": "you have",
    "we've": "we have",
    "they've": "they have",
    "i'll": "I will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "can't": "cannot",
    "couldn't": "could not",
    "won't": "will not",
    "wouldn't": "would not",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "let's": "let us",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "there's": "there is",
    "here's": "here is",
    "shouldn't": "should not",
    "mustn't": "must not",
    "shan't": "shall not"
}

# Function to expand contractions
def expand_contractions(text):
    contractions_pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b', flags=re.IGNORECASE)
    def replace(match):
        return contractions_dict.get(match.group(0).lower(), match.group(0))
    return contractions_pattern.sub(replace, text)


In [15]:
# Define a set of informal tokens to remove
informal_tokens = {
    "u": "you",
    "r": "are",
    "lmao": "",  # Remove
    "lol": "",   # Remove
    "btw": "by the way",
    "idk": "I do not know",
    "omg": "oh my god",
    "gonna": "going to",
    "wanna": "want to",
    "gotta": "got to",
    "b/c": "because",
    "thx": "thanks",
    "pls": "please",
    "cuz": "because",
    "wut": "what",
    "smh": "",  # Remove
    "k": "okay",
    "ttyl": "talk to you later"
}

def remove_informal_tokens(text):
    # Replace informal tokens
    for token, replacement in informal_tokens.items():
        # Use regex to match whole words and replace them
        text = re.sub(r'\b' + re.escape(token) + r'\b', replacement, text, flags=re.IGNORECASE)
    return text

def fix_punctuation(text):
    # Ensure proper spacing after punctuation
    text = re.sub(r'\s*([.,;:!?()])\s*', r'\1 ', text)  # Ensure space after punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Trim leading and trailing spaces
    return text

# Combined function to clean the text
def clean_text(text):
    text = remove_informal_tokens(text)  # Remove informal tokens
    text = fix_punctuation(text)  # Fix punctuation
    return text

def preprocess_ellipses(text):
    # Replace occurrences of three or more dots with actual ellipses
    text = re.sub(r'(\.\s*){3,}', '...', text)
    return text




In [16]:
import nltk
from nltk.corpus import stopwords

# Download necessary resources
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = load_text(text)              # Load text from file path
    ##################################################################################################################
    ### Comment the following points until return out, to simulate the bare processing without normalization steps ###
    ##################################################################################################################
    text = expand_contractions(text)    # Expand contractions
    text = remove_informal_tokens(text) # Replace informal tokens
    text = ' '.join(word for word in text.split() if word not in stop_words) # Remove stopwords
    text = fix_punctuation(text)        # Fix punctuation
    text = preprocess_ellipses(text)    # Handle ellipses as a single token
    text = re.sub(r'<.*?>', '', text)   # Remove HTML tags
    return text


# Apply preprocessing function to training and test data
data_train['cleaned_text'] = data_train['path'].apply(preprocess_text)
data_test['cleaned_text'] = data_test['path'].apply(preprocess_text)

# Number of samples to be printed
n_samples = 5

# Randomly sample indices from "data_train"
sample_indices = random.sample(range(len(data_train)), n_samples)

# Print each sample from the cleaned_text
for i, idx in enumerate(sample_indices, start=1):
    # Access the cleaned text directly
    text = data_train.loc[idx, 'cleaned_text']
    print(f"Sample {i}:\n")
    print(text)
    print("\n" + "="*80 + "\n")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kentf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Sample 1:

This one worst films I ever seen. I looked mainly morbid curiosity since I loved novel, I wish I not. I turned little less hour, though I wanted turn five minutes. I wish I had. It disregards novel lot changes sorts factors. Unless film managed redeem last 50 minutes( which would impossible) I would way recommend this. Its insult one greatest writers 20th century. I think, many people say is, "The Bell Jar" necessarily unfilmable, particular rendition could done without. I'd almost like see one day hands director screenwriter justice.


Sample 2:

This film renewed interest French cinema. The story enchanting, acting flawless Audrey Tautou absolutely beautiful. I imagine seeing lot States upcoming role Amelie.


Sample 3:

Superman II - The Richard Donner Cut fan's dream come true. At long last, footage seen photos scenes existed printed page would finally come life. A director unable complete vision would opportunity vision restored. It seems like winning situation. And sta

### A simple pipeline:
[Do not change it!]

This simple pipeline will be used to compare the newly created pipeline against, to evaluate the performance increase.

**White-Space tokenization:**

In [17]:
def tokenize(text:str):
    ''' An example tokenization function. '''

    # simple white-space tokenization:
    return text.lower().split()


**Bag-of-words Embedding:**

See documentation of [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# create a simple bag of words embedding:
bow = CountVectorizer(

    # the next line converts the filepaths to the actual texts:
    #preprocessor = load_text, # Commented, to check if the preprocessing from above changes the results

    # tokenization function from above:
    tokenizer = tokenize,

    # Set token_pattern to None since we're using a custom tokenizer
    token_pattern=None

)

# Train the embedding on cleaned training data
embeddings_train = bow.fit_transform(data_train['cleaned_text'].values)

# Vectorize the cleaned test data
embeddings_test = bow.transform(data_test['cleaned_text'].values)

# These are the original lines
# train the embedding:
#embeddings_train = bow.fit_transform(data_train['path'].values)

# vectorize test data:
#embeddings_test = bow.transform(data_test['path'].values)

**Classification with a linear SVM**

See documentation of [sklearn.svm.LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

In [19]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

svm = LinearSVC(dual=False, max_iter=5000)

# train classifier:
svm.fit(embeddings_train, data_train['label'].values)

# test classifier:
predictions = svm.predict(embeddings_test)

# Calculate Accuracy:
print('Accuracy:', accuracy_score(data_test['label'].values, predictions))

Accuracy: 0.792


### Own text mining pipeline

As the text was previously normalized / or not in the other case, the next step is to tokenize the text.
For this, we are employing the tokenizer with BPE.

In [26]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize a BPE tokenizer
tokenizer = Tokenizer(BPE())

# Add a pre-tokenizer to handle whitespace properly
tokenizer.pre_tokenizer = Whitespace()

# Initialize a BPE trainer with some default parameters
trainer = BpeTrainer(special_tokens=["<pad>", "<s>", "</s>", "<unk>", "__ellipsis__"])

# Convert cleaned_text column to a list of strings for training
training_data = data_train['cleaned_text'].tolist()

# Train the tokenizer on the cleaned text data
tokenizer.train_from_iterator(training_data, trainer)

# Save the tokenizer for later use
tokenizer.save("imdb_bpe_tokenizer.json")

# Encode the text data for the model
encoded_train = [tokenizer.encode(text).ids for text in data_train['cleaned_text']]
encoded_test = [tokenizer.encode(text).ids for text in data_test['cleaned_text']]

# Select a single example to decode and print
sample_index = 0  # Choose an index, e.g., 0 for the first item
encoded_sample = encoded_train[sample_index]

# Decode the tokenized output back to text
decoded_text = tokenizer.decode(encoded_sample)

# Select additional examples to decode and print
sample_indices = [0, 6, 12]  # You can adjust these indices as needed

for i, sample_index in enumerate(sample_indices):
    encoded_sample = tokenizer.encode(data_train['cleaned_text'][sample_index])
    decoded_text = tokenizer.decode(encoded_sample.ids)
    
    print(f"Sample {i+1} Original string:", data_train['cleaned_text'][sample_index])  # Original text for comparison
    print(f"Sample {i+1} Encoded tokens (token IDs):", encoded_sample.ids)  # Encoded token IDs
    print(f"Sample {i+1} Tokenized string (tokens):", encoded_sample.tokens)  # Encoded tokens as text
    print(f"Sample {i+1} Decoded string:", decoded_text)  # Decoded string
    print("="*80)

Sample 1 Original string: Bromwell High cartoon comedy. It ran time programs school life, "Teachers". My 35 years teaching profession lead believe Bromwell High's satire much closer reality "Teachers". The scramble survive financially, insightful students see right pathetic teachers' pomp, pettiness whole situation, remind schools I knew students. When I saw episode student repeatedly tried burn school, I immediately recalled...High. A classic line: INSPECTOR: I sack one teachers. STUDENT: Welcome Bromwell High. I expect many adults age think Bromwell High far fetched. What pity not!
Sample 1 Encoded tokens (token IDs): [9284, 2818, 2984, 834, 18, 278, 2906, 272, 11627, 1693, 431, 16, 6, 13956, 751, 919, 6442, 509, 14376, 6967, 689, 773, 9284, 2818, 11, 82, 8960, 344, 7050, 2554, 6, 13956, 751, 157, 26485, 4983, 11223, 16, 9165, 4904, 263, 511, 2502, 9156, 11, 8109, 16, 21461, 837, 1726, 16, 1777, 13916, 43, 2656, 4904, 18, 908, 43, 633, 698, 4305, 6370, 2094, 2258, 1693, 16, 43, 3873,

#### Analysis of the results

To be discussed

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer with desired parameters
tfidf_vectorizer = TfidfVectorizer(
    max_features=15000,    
    ngram_range=(1, 2),
)

# Fit on training data and transform both training and test data
tfidf_train = tfidf_vectorizer.fit_transform(data_train['cleaned_text'])
tfidf_test = tfidf_vectorizer.transform(data_test['cleaned_text'])

# Print the shapes of the resulting TF-IDF matrices
print("TF-IDF Training Data Shape:", tfidf_train.shape)
print("TF-IDF Test Data Shape:", tfidf_test.shape)


TF-IDF Training Data Shape: (1000, 15000)
TF-IDF Test Data Shape: (1000, 15000)


In [22]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the SVM classifier
svm = LinearSVC()
svm.fit(tfidf_train, data_train['label'])  # 'label' is your target variable

# Make predictions on the test data
predictions = svm.predict(tfidf_test)

# Evaluate the classifier's performance
accuracy = accuracy_score(data_test['label'], predictions)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(data_test['label'], predictions))

Accuracy: 0.819

Classification Report:
               precision    recall  f1-score   support

         neg       0.81      0.83      0.82       500
         pos       0.83      0.80      0.82       500

    accuracy                           0.82      1000
   macro avg       0.82      0.82      0.82      1000
weighted avg       0.82      0.82      0.82      1000



c:\Users\kentf\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


#### Try another Predicition Model on the same tokenized (and normalized) data.

A neural network is trained with the same data as above to see if that might perform better on the same preprocessed vectors than the svm.

In [23]:
import torch
print(torch.__version__)          # To confirm the version
print(torch.cuda.is_available())  # This should return False as there is no GPU

2.4.1+cpu
False


In [24]:
# Map the labels to integers
label_mapping = {'pos': 1, 'neg': 0}
data_train['label'] = data_train['label'].map(label_mapping)
data_test['label'] = data_test['label'].map(label_mapping)

# Now convert to tensor
y_train_tensor = torch.tensor(data_train['label'].values, dtype=torch.long)
y_test_tensor = torch.tensor(data_test['label'].values, dtype=torch.long)

In [25]:
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import TensorDataset, DataLoader

# Convert to dense tensors
X_train_tensor = torch.tensor(tfidf_train.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(tfidf_test.toarray(), dtype=torch.float32)

# Labels as tensors
y_train_tensor = torch.tensor(data_train['label'].values, dtype=torch.long)
y_test_tensor = torch.tensor(data_test['label'].values, dtype=torch.long)

# Define the Neural Network model
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# Initialize the model
input_size = X_train_tensor.shape[1]
model = SimpleNN(input_size)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training Loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}')

# Evaluation on Test Data
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = accuracy_score(y_test_tensor, predicted)
    print("Test Accuracy:", accuracy)
    print("\nClassification Report:\n", classification_report(y_test_tensor, predicted))


Epoch 1/2, Loss: 0.6120333722792566
Epoch 2/2, Loss: 0.06782460274916957
Test Accuracy: 0.813

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.83      0.82       500
           1       0.82      0.80      0.81       500

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000

